<a href="https://colab.research.google.com/github/ayush-verma-15/GenAI-/blob/main/Ass2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
english_sentences = ['hello', 'how are you', 'good morning', 'thank you', 'i love you']
french_sentences = ['bonjour', 'comment ça va', 'bonjour', 'merci', 'je t\'aime']

input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(english_sentences)
input_sequences = input_tokenizer.texts_to_sequences(english_sentences)
input_tensor = pad_sequences(input_sequences, padding='post')

target_tokenizer = Tokenizer(filters='')
target_tokenizer.fit_on_texts(['<start> ' + t + ' <end>' for t in french_sentences])
target_sequences = target_tokenizer.texts_to_sequences(['<start> ' + t + ' <end>' for t in french_sentences])
target_tensor = pad_sequences(target_sequences, padding='post')

# Parameters
num_encoder_tokens = len(input_tokenizer.word_index) + 1
num_decoder_tokens = len(target_tokenizer.word_index) + 1
embedding_dim = 256
units = 512

# Prepare decoder inputs and outputs
target_tensor_input = target_tensor[:, :-1]
target_tensor_output = target_tensor[:, 1:]


In [4]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(input_dim=num_encoder_tokens, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb = Embedding(input_dim=num_decoder_tokens, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Full model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 256) │      2,560 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 256) │      2,560 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 512),     │  1,574,912 │ embedding[0][0]   │
│                     │ (None, 512),      │            │                   │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │  1,574,912 │ embedding_1[0][0… │
│                     │ 512), (None,      │            │ lstm[0][1],       │
│                     │ 512), (None,      │            │ lstm[0][2]        │
│                     │ 512)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 10)  │      5,130 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,160,074 (12.05 MB)

 Trainable params: 3,160,074 (12.05 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
target_tensor_output = np.expand_dims(target_tensor_output, -1)

history = model.fit([input_tensor, target_tensor_input], target_tensor_output,
                    batch_size=2, epochs=10)


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 128ms/step - loss: 2.2908
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 2.1186
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 1.9154
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 1.5383
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - loss: 1.3047
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - loss: 1.1498
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - loss: 1.3053
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - loss: 1.0525
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 0.9866
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - loss: 0.8024


In [6]:
# Compile the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

target_tensor_output = np.expand_dims(target_tensor_output, -1)

# Train the model
model.fit([input_tensor, target_tensor_input], target_tensor_output,
          batch_size=64, epochs=10)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 0.9563
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - loss: 0.9006
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step - loss: 0.8415
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - loss: 0.8162
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step - loss: 0.7477
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - loss: 0.7100
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step - loss: 0.6727
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step - loss: 0.6421
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step - loss: 0.6137
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - loss: 0.5891


In [7]:
model.save("seq2seq_model.h5")

In [8]:
from tensorflow.keras.models import load_model
model = load_model("seq2seq_model.h5")